In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectPercentile
from sklearn.metrics import classification_report, accuracy_score, recall_score
import pandas as pd

In [2]:
# Initialize feature and target datasets
X = pd.read_csv(r"C:\Programming\Python\Industry Level Projects\Django E-t-E Project\myproject\notebooks\features.csv")
y = pd.read_csv(r"C:\Programming\Python\Industry Level Projects\Django E-t-E Project\myproject\notebooks\target.csv")

X.head()

,age,marital,credit_on_default,annual_balance,housing_loan,personal_loan,contact_day,contact_duration,campaign_contacts,days_last_contacted,...,contact_month_dec,contact_month_feb,contact_month_jan,contact_month_jul,contact_month_jun,contact_month_mar,contact_month_may,contact_month_nov,contact_month_oct,contact_month_sep
0,59,1,0,2343,1,0,5,1042,1,-1,...,False,False,False,False,False,False,True,False,False,False
1,56,1,0,45,0,0,5,1467,1,-1,...,False,False,False,False,False,False,True,False,False,False
2,41,1,0,1270,1,0,5,1389,1,-1,...,False,False,False,False,False,False,True,False,False,False
3,55,1,0,2476,1,0,5,579,1,-1,...,False,False,False,False,False,False,True,False,False,False
4,54,1,0,184,0,0,5,673,2,-1,...,False,False,False,False,False,False,True,False,False,False


In [12]:
# Split data into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25, random_state=42)

# Train demo model to get feature importances
demo_model = RandomForestClassifier(random_state=42)
demo_model.fit(X_train, y_train)
feature_importances = demo_model.feature_importances_

# Map importances to their features
feature_importance_df = pd.DataFrame({
    "Feature" : X.columns,
    'Importance' : feature_importances
})

feature_importance_df.head()

C:\Users\LESEGO\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


,Feature,Importance
0,age,0.078876
1,marital,0.013463
2,credit_on_default,0.001496
3,annual_balance,0.080912
4,housing_loan,0.025059


In [23]:
feature_importance_df.sort_values(by='Importance', ascending=False, inplace=True)  # Sort features by most important

selected_features = list(feature_importance_df['Feature'][0:5])  # Put selected features into list

feature_importance_df.head()

,Feature,Importance
7,contact_duration,0.352638
3,annual_balance,0.080912
0,age,0.078876
6,contact_day,0.070257
11,poutcome,0.044352


In [24]:
selected_features

['contact_duration', 'annual_balance', 'age', 'contact_day', 'poutcome']

In [5]:


# Scale X data for better model performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Assert that the scaled data has the same shape as the original values
assert(X_train.shape == X_train_scaled.shape)
assert(X_test.shape == X_test_scaled.shape)

,age,marital,credit_on_default,annual_balance,housing_loan,personal_loan,contact_day,contact_duration,campaign_contacts,days_last_contacted,...,contact_month_dec,contact_month_feb,contact_month_jan,contact_month_jul,contact_month_jun,contact_month_mar,contact_month_may,contact_month_nov,contact_month_oct,contact_month_sep
5304,60,1,0,8121,1,1,18,98,7,-1,...,False,False,False,False,False,False,False,False,False,False
7512,51,0,1,-55,1,0,11,281,2,266,...,False,False,False,False,False,False,True,False,False,False
4700,28,0,1,-1042,1,0,6,712,2,-1,...,False,False,False,False,True,False,False,False,False,False
4918,30,1,0,4457,1,0,21,566,4,-1,...,False,False,False,False,False,False,False,True,False,False
4450,31,0,0,2544,0,0,15,252,1,184,...,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,47,1,0,761,1,0,11,80,2,-1,...,False,False,False,True,False,False,False,False,False,False
5191,28,0,0,159,0,0,16,449,2,33,...,False,False,False,False,False,False,False,True,False,False
5390,35,1,0,1144,0,0,20,197,13,-1,...,False,False,False,False,False,False,False,False,False,False
860,51,1,0,746,0,0,25,372,5,-1,...,False,False,False,False,False,False,False,False,False,False


# Feature Engineering

In [47]:
print(f"Amount of features before selection:\t{X_train_scaled.shape[1]}")

# Initialize feature selector
select = SelectPercentile(percentile=50)
select.fit(X_train_scaled, y_train)

# Select features from train and test
X_train_selected = select.transform(X_train_scaled)
X_test_selected = select.transform(X_test_scaled)

print(f"Amount of features after selection:\t{X_train_selected.shape[1]}")

Amount of features before selection:	39
Amount of features after selection:	19


C:\Users\LESEGO\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\validation.py:1310: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# HyperParameter Tuning

In [ ]:
param_grid = {
    
}
grid_search = GridSearchCV()

# Training

# Comparing Models